## Description: This script

Input:   data/ref_data/ARanalysis_ref_data.nc

       'data/AR_features/part1/AR_feature.part1.%s.1981-2015.nc' % (method)

Output: 'data/AR_features/part2/AR_feature.part2.%s.1981-2015.nc' % (method)

In [3]:
import numpy as np
import netCDF4 as nc
from skimage.draw import polygon
from skimage import measure
from math import radians, sin, cos, acos

import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
rootdir = '/raid1/chen423/serdp/archive/GRL2018/'

In [5]:
ref_file = rootdir+'data/ref_data/ARanalysis_ref_data.nc'
refgroup = nc.Dataset(ref_file, 'r', format='NETCDF4')
ref_lats = refgroup.variables['pt_lat'][:]
ref_lons = refgroup.variables['pt_lon'][:]
refgroup.close()

In [6]:
def calc_grid_length(index):
    
    slat = radians(float((ref_lats[index]+ref_lats[index-1])/2))
    slon = radians(float((ref_lons[index]+ref_lons[index-1])/2))
    elat = radians(float((ref_lats[index]+ref_lats[index+1])/2))
    elon = radians(float((ref_lons[index]+ref_lons[index+1])/2))
    dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    return dist

In [7]:
# construct reference distance
ref_gridlen = np.zeros(ref_lats.shape)
for i in np.arange(1,ref_gridlen.shape[0]-1):
    ref_gridlen[i] = calc_grid_length(i)
    
# first point
slat = radians(float(ref_lats[0]))
slon = radians(float(ref_lons[0]))
elat = radians(float((ref_lats[0]+ref_lats[1])/2))
elon = radians(float((ref_lons[0]+ref_lons[1])/2))
ref_gridlen[0] = (6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon)))*2

# last point
slat = radians(float(ref_lats[ref_gridlen.shape[0]-2]))
slon = radians(float(ref_lons[ref_gridlen.shape[0]-2]))
elat = radians(float((ref_lats[ref_gridlen.shape[0]-2]+ref_lats[ref_gridlen.shape[0]-1])/2))
elon = radians(float((ref_lons[ref_gridlen.shape[0]-2]+ref_lons[ref_gridlen.shape[0]-1])/2))
ref_gridlen[ref_gridlen.shape[0]-1]= (6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon)))*2

In [8]:
def calc_ARcentroid(xs, ys):
    weights = ref_gridlen[ys]*ARint_coast[xs,ys]
    centroid_x = ((xs*weights).sum())/(weights.sum())
    centroid_y = ((ys*weights).sum())/(weights.sum())
    return int(round(centroid_x)), int(round(centroid_y))

In [9]:
def verify_AR(incontour, ARint_coast):
    xs, ys = polygon(np.round_(incontour[:,0]), np.round_(incontour[:,1]), ARint_coast.shape)
    # sometimes the contour denotes the "missing" part within a larger AR contour, so get rid of these.
    weights = ref_gridlen[ys]*ARint_coast[xs,ys]
    if weights.sum()==0:
        sig = False
    else:
        sig = True
    return sig

In [10]:
def calc_2pts_distance(sindex, eindex):
    
    slat = radians(float(ref_lats[sindex]))
    slon = radians(float(ref_lons[sindex]))
    elat = radians(float(ref_lats[eindex]))
    elon = radians(float(ref_lons[eindex]))
    #print("(%.3f  %.3f), (%.3f  %.3f)" % (ref_lats[sindex], ref_lons[sindex], ref_lats[eindex], ref_lons[eindex]))

    dist = 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))
    return dist

In [11]:
def compute_event_coastal_feature(x_sets, y_sets):
    # x is time dimension, y is space dimension
    
    # mean width along the coast, and mean intensity at the coast
    total_width = 0
    count = 0
    total_intensity = 0
    total_intensity_count = 0
    for x in np.unique(x_sets):
        tmp_ys = y_sets[x_sets==x]
        tmp_ymin = tmp_ys.min()
        tmp_ymax = tmp_ys.max()
        #print(x, tmp_ymin, tmp_ymax)
        if tmp_ymax==tmp_ymin:
            total_width = total_width + ref_gridlen[tmp_ymax]
        else:
            total_width = total_width + calc_2pts_distance(tmp_ymin, tmp_ymax)
        for y in np.unique(tmp_ys):
            total_intensity = total_intensity + ARint_coast[x,y] * ref_gridlen[y]
            total_intensity_count = total_intensity_count + ref_gridlen[y]
        count = count + 1
    mean_width = total_width/float(count)
    mean_intensity = total_intensity/total_intensity_count
    
    return mean_width, mean_intensity

In [12]:
# one of the key functions. It will correct the grids in the polygon, by removing those where coast intensity is zero. this is
# caused by the interpolation of measure.find_contours.
def clip_out_real_AR(inx, iny, coast_int):
    outx = []
    outy = []
    for i in np.arange(inx.shape[0]):
        if coast_int[inx[i], iny[i]]>0:
            outx.append(inx[i])
            outy.append(iny[i])
    outx_array = np.asarray(outx)
    outy_array = np.asarray(outy)
    return outx_array, outy_array

In [13]:
def compute_ARevent_features(incontour, refdata):
    
    xs_raw, ys_raw = polygon(np.round_(incontour[:,0]), np.round_(incontour[:,1]), refdata.shape)
    xs, ys = clip_out_real_AR(xs_raw, ys_raw, ARint_coast)
    outdata = np.ones(11)*9999.0
    
    # following values are already weight by gridarea at every timestep. So here just take one value per timestep, then average.
    # 1. area, full
    tmp_total = 0
    tmp_count = 0
    for i in np.unique(xs):
        tmp_total = tmp_total + ARarea_full[i,ys[xs==i][0]]
        tmp_count = tmp_count + 1
    outdata[0] = tmp_total/float(tmp_count)
    
    # 2. area, land
    tmp_total = 0
    tmp_count = 0
    for i in np.unique(xs):
        tmp_total = tmp_total + ARarea_land[i,ys[xs==i][0]]
        tmp_count = tmp_count + 1
    outdata[1] = tmp_total/float(tmp_count)
    
    # 3. intensity, full area
    tmp_total = 0
    tmp_count = 0
    for i in np.unique(xs):
        tmp_total = tmp_total + ARint_full[i,ys[xs==i][0]]*ARarea_full[i,ys[xs==i][0]]
        tmp_count = tmp_count + ARarea_full[i,ys[xs==i][0]]
    outdata[2] = tmp_total/float(tmp_count)
    
    # 4. intensity, land
    tmp_total = 0
    tmp_count = 0
    for i in np.unique(xs):
        tmp_total = tmp_total + ARint_land[i,ys[xs==i][0]]*ARarea_land[i,ys[xs==i][0]]
        tmp_count = tmp_count + ARarea_land[i,ys[xs==i][0]]
    outdata[3] = tmp_total/float(tmp_count)
    
    # 5. coastal features
    # coastal width;  coastal intensity
    outdata[4], outdata[5] = compute_event_coastal_feature(xs, ys)    
    
    # 6. centroid
    centroid_x, centroid_y = calc_ARcentroid(xs, ys)
    outdata[6] = xs.min()  # start timestep
    outdata[7] = (xs.max()-xs.min()+1)*3   # duration, in hours
    # find centroid time and location
    outdata[8] = centroid_x  # center timestep
    outdata[9] = ref_lats[centroid_y]  # center lat
    outdata[10] = ref_lons[centroid_y]  # center lon

    return outdata

In [29]:
def save_nc_results(method, outdata):
    
    outfile = rootdir+'data/AR_features/part2/%s.AR_events_feature.1981-2015.nc' % (method)
    outgroup = nc.Dataset(outfile, 'w', format='NETCDF4')

    # dimension
    edim = outgroup.createDimension('event', outdata.shape[0])
    vdim = outgroup.createDimension('feature', 11)

    # vars
    evar = outgroup.createVariable('event', 'i4', ('event'))
    evar.long_name = 'AR events reaching US west coast'
    vvar = outgroup.createVariable('feature', 'i4', ('feature'))
    vvar.long_name = 'AR features'
    evar[:] = np.arange(outdata.shape[0])
    vvar[:] = np.arange(11)

    featurevar = outgroup.createVariable('AR_event_feature', 'f4', ('event','feature'))
    featurevar[:] = outdata
    featurevar.feature0 = 'mean size of whole AR region. Only counting those times where AR intersects with coast/land'
    featurevar.feature0unit = 'km^2'
    featurevar.feature1 = 'mean size of AR region over land'
    featurevar.feature1unit = 'km^2'
    featurevar.feature2 = 'mean intensity of whole AR'
    featurevar.feature2unit = 'kg/m/s'
    featurevar.feature3 = 'mean intensity of land AR'
    featurevar.feature3unit = 'kg/m/s'
    featurevar.feature4 = 'coastal width of AR'
    featurevar.feature4unit = 'km'
    featurevar.feature5 = 'coastal mean intensity of AR'
    featurevar.feature5unit = 'kg/m/s'
    featurevar.feature6 = 'start timesetp of AR'
    featurevar.feature6unit = 'index. Step 0 is 1981-01-01_00, deltatime is 3 hours'
    featurevar.feature7 = 'duration of AR. Only counting those time when AR intersects with coast/land'
    featurevar.feature7unit = 'hours'
    featurevar.feature8 = 'AR center timestep'
    featurevar.feature8unit = 'index. Step 0 is 1981-01-01_00, deltatime is 3 hours'
    featurevar.feature9 = 'AR center latitude'
    featurevar.feature9unit = 'degrees_north'
    featurevar.feature10 = 'AR center longitude'
    featurevar.feature10unit = 'degrees_east'

    outgroup.script = '/usr1/chen423/serdp/tools/ipython/paper1_experiments/Regression_with_AR/Step2_collect_ARfeature.part2_events.ipynb'

    outgroup.close()

## main function

for 5 methods. For "rutz", see the followig block.

In [23]:
#method = 'gershunov'
#method = 'goldenson'
#method = 'guan'
#method = 'pnnl1'
#method = 'pnnl2'

#method = 'lora'
#method = 'payne'
method = 'shields'
#method = 'tempest'
#method = 'walton'

infile = rootdir+'data/AR_features/part1/AR_feature.part1.%s.1981-2015.nc' % (method)
ingroup = nc.Dataset(infile, 'r', format='NETCDF4')
ARint_full = ingroup.variables['ARint_full'][:]
ARint_land = ingroup.variables['ARint_land'][:]
ARint_coast = ingroup.variables['ARint_coast'][:]
ARarea_full = ingroup.variables['ARarea_full'][:]
ARarea_land = ingroup.variables['ARarea_land'][:]
ingroup.close()
    
refdata = ARint_coast.copy()
refdata[ARint_coast>0] = 1
contours = measure.find_contours(refdata, 0.9)
    
    
nevents_fake = len(contours)
AR_TF = np.zeros(nevents_fake)
for i in np.arange(nevents_fake):
    sig = verify_AR(contours[i], ARint_coast)
    if sig==True:
        AR_TF[i] = 1

nevents = (AR_TF==1).sum()
print(nevents_fake, nevents)
    
outdata_full = np.ones((nevents,11))*9999.0
count = 0
for i in np.arange(nevents_fake):
    if AR_TF[i]==1:
        outdata_full[count,:] = compute_ARevent_features(contours[i], refdata)
        count = count + 1
            
#save_nc_results(method, outdata_full)

KeyboardInterrupt: 

## the codes below are for rutz only.

There are some events whose main region is in the very south of 25-30 band, so the AR intesity at coast are pretty low (i.e., <100kg/m/s).

In [14]:
method = 'rutz'

infile = rootdir+'data/AR_features/part1/AR_feature.part1.%s.1981-2015.nc' % (method)
ingroup = nc.Dataset(infile, 'r', format='NETCDF4')
ARint_full = ingroup.variables['ARint_full'][:]
ARint_land = ingroup.variables['ARint_land'][:]
ARint_coast = ingroup.variables['ARint_coast'][:]
ARarea_full = ingroup.variables['ARarea_full'][:]
ARarea_land = ingroup.variables['ARarea_land'][:]
ingroup.close()
    
refdata = ARint_coast.copy()
refdata[ARint_coast>0] = 1
contours = measure.find_contours(refdata, 0.9)
    
    
nevents_fake = len(contours)
AR_TF = np.zeros(nevents_fake)
for i in np.arange(nevents_fake):
    sig = verify_AR(contours[i], ARint_coast)
    if sig==True:
        AR_TF[i] = 1

nevents = (AR_TF==1).sum()
print(nevents_fake, nevents)
    
outdata_full = np.ones((nevents,11))*9999.0
count = 0
for i in np.arange(nevents_fake):
    if AR_TF[i]==1:
        outdata_full[count,:] = compute_ARevent_features(contours[i], refdata)
        count = count + 1

# remove the strange events
test_data = outdata_full[:,5]
events_use = (test_data>200).sum()
outdata_full_clip = np.zeros((events_use, 11))

count = 0
for i in np.arange(test_data.shape[0]):
    if outdata_full[i,5]>200:
        outdata_full_clip[count,:] = outdata_full[i,:]
        count = count + 1
    
#save_nc_results(method, outdata_full_clip)

12014 7328


In [15]:
print(outdata_full.shape, events_use)

(7328, 11) 7324


## debug

In [ ]:
def trace_event_to_orig(inevent):
    count = 0
    for i in np.arange(AR_TF.shape[0]):
        if AR_TF[i]==1:
            count = count + 1
            if count==inevent+1:
                print('%d -> %d' % (count-1, i))

In [ ]:
def translate_results(indata):
    print('AR area:                %.3f km^2' % (indata[0]))
    print('AR area (land):         %.3f km^2' % (indata[1]))
    print('AR intensity:           %.3f kg/m/s' % (indata[2]))
    print('AR intensity (land):    %.3f kg/m/s' % (indata[3]))
    print('AR width (coastal):     %.3f km' % (indata[4]))
    print('AR intensity (coastal): %.3f kg/m/s' % (indata[5]))
    print('start time:             %d index' % (indata[6]))
    print('duration:               %d hours' % (indata[7]))
    print('center time:            %d index' % indata[8])
    print('center lat:             %.3f deg' % (indata[9]))
    print('center lon:             %.3f deg' % (indata[10]))

In [ ]:
np.argwhere(outdata_full[:,7]==234)